# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4836, uptime 0:00:34
mini_beamline                    RUNNING   pid 4837, uptime 0:00:34
random_walk                      RUNNING   pid 4838, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4839, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4840, uptime 0:00:34
simple                           RUNNING   pid 4841, uptime 0:00:34
thermo_sim                       RUNNING   pid 4842, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4843, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-03 20:05:32
Persistent Unique Scan ID: 'cde34928-4c32-4388-813e-69df0a62d57e'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 20:05:32.5 |     -1.000 |     102306 |
|         2 | 20:05:32.5 |     -0.500 |     103599 |
|         3 | 20:05:32.6 |      0.000 |     104324 |
|         4 | 20:05:32.6 |      0.500 |     104655 |
|         5 | 20:05:32.6 |      1.000 |     102758 |


+-----------+------------+------------+------------+
generator scan ['cde34928'] (scan num: 3)





('cde34928-4c32-4388-813e-69df0a62d57e',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/cde34928-4c32-4388-813e-69df0a62d57e-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-03 20:05:32
Persistent Unique Scan ID: 'b5530330-b7af-4be8-8a0c-41a36ea61ba3'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 20:05:33.1 |          -1 | 1130799213 |


|         2 | 20:05:33.2 |          -0 | 1128885455 |


|         3 | 20:05:33.3 |           0 | 1122486789 |


|         4 | 20:05:33.4 |           0 | 1117689681 |


|         5 | 20:05:33.5 |           1 | 1112147057 |


+-----------+------------+-------------+------------+
generator scan ['b5530330'] (scan num: 4)





('b5530330-b7af-4be8-8a0c-41a36ea61ba3',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-03 20:05:33
Persistent Unique Scan ID: '05a051e9-c7e2-4e04-b6e2-b45d94cee8ba'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 20:05:34.0 | 1088242717 |


|         2 | 20:05:34.9 | 1053772083 |


|         3 | 20:05:35.9 | 1099239350 |


...

|        28 | 20:06:01.0 | 1130626971 |


|        29 | 20:06:02.0 | 1082794408 |


|        30 | 20:06:03.0 | 1054549209 |


+-----------+------------+------------+
generator count ['05a051e9'] (scan num: 5)





('05a051e9-c7e2-4e04-b6e2-b45d94cee8ba',)